In [241]:
import os
import pandas as pd

#set directories
ASAP_DIR = r'/home/levi/asap'
SILKROAD_DIR = r'/home/levi/silkroad'
VICE_DIR = r'/home/levi/vice_city'
DARKFOX_DIR = r'/home/levi/darkfox'


def get_names(DIR):
    '''retrieve vendor names from html file in directories'''
    
    files = os.listdir(DIR)
    vendor_names = [i[:-5] for i in files]
    
    return vendor_names

#get vendor names for all marketplaces
asap_vendors = get_names(ASAP_DIR)
silkroad_vendors = get_names(SILKROAD_DIR)
vice_vendors = get_names(VICE_DIR)
darkfox_vendors = get_names(DARKFOX_DIR)

all_vendors = darkfox_vendors + silkroad_vendors + vice_vendors + asap_vendors
all_vendors = list(dict.fromkeys(all_vendors)) #remove duplicates

In [242]:
#df.loc[df['name'] == 'DopeNL', 'products'] = 10

#list of common marketplaces
list_of_markets = ['apollon', 'white house', 'vice city', 'torrez', 'empire', 'silkroad', 'darkfox','asap',
                   'berlusconi', 'cartel','monopoly','icarus','pax romana', 'cryptonia', 'nightmare', 
                   'invictus', 'world market', 'traderoute', 'wall street', 'versus', 'cannazon', 'dark0de',
                  'tor2door','olympus']

In [246]:
###SCRAPE DATA FROM DARKFOX####
from bs4 import BeautifulSoup as bs


darkfox_data=[] #empty list which will contain our data


for i in range(len(os.listdir(DARKFOX_DIR))): #for each html file
    
    darkfox_df = {} #empty dictionary which will contain user data
    
    categories = []
    completed_orders = [0]
    ships_from = []
    active_markets = []
    
    
    name = os.listdir(DARKFOX_DIR)[i][:-5] #vendor name
      
    
    with open(DARKFOX_DIR+'/' + os.listdir(DARKFOX_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
    #find active markets
    for market in list_of_markets:
        if market in soup.text.lower(): #lowercase text
            active_markets.append(market)
    
        
    #find product categories    
    for line in list(txt):
        if 'Category' in line:
            categories.append(line[10:])
        if 'Ships from' in line:
            ships_from.append(line[12:])
            
            
    #remove duplicates  
    categories = list(dict.fromkeys(categories))
    ships_from = list(dict.fromkeys(ships_from))
    active_markets = list(dict.fromkeys(active_markets))
    
    other_orders = 0
    #find number of completed orders
    for idx, line in enumerate(list(txt)): 
            if 'Completed orders:' in line:
                orders = int(txt[idx+1])
                completed_orders.append(orders)
            if 'marketplace' in line: 
            
                other_orders =  int(line[:-27])
            
    
   
    #add name, categories,completed orders, location and known markets
    darkfox_df['name'] = name
    darkfox_df['products'] = categories
    darkfox_df['completed_orders'] = int(completed_orders[-1])#there are two numbers so only add the last one
    darkfox_df['other_orders'] = other_orders
    darkfox_df['ships_from'] = ships_from
    #darkfox_df['active_markets'] = active_markets
    
    darkfox_data.append(darkfox_df) #add user to dataframe
        

#transform to dataframe
darkfox_df = pd.DataFrame(data= darkfox_data)

In [247]:
#adding up orders
darkfox_df["orders"] = (darkfox_df["completed_orders"] + darkfox_df["other_orders"]).astype("int")
darkfox_df= darkfox_df[['name', 'products', 'orders','ships_from']]

darkfox_df.to_csv('data/darkfox_data.csv')

darkfox_df.sort_values('orders', ascending =False)
## final df contains, name, products, orders and location

,name,products,orders,ships_from
402,monoko,"[Stimulants, Benzos, Weight Loss, Drugs]",57013,[United Kingdom]
105,terrysukstock,[Benzos],53751,[United Kingdom]
191,DreamWeaver,"[Amphetamine, Weed]",29819,[United Kingdom]
347,DrRelax,"[Weed, Amphetamine, Stimulants, Cocaine]",21921,[United Kingdom]
109,tomandjerry,"[Heroin, Ketamine, Crack, MDMA, Cocaine, LSD]",19463,[Netherlands]
...,...,...,...,...
386,colombiaconnec,[],0,[]
161,greatwhite,[],0,[]
559,MassaLiaPan,"[Hacking, Malware, Fraud]",0,[]
558,accountmaster,[],0,[]


In [248]:
###ASAP SCRAPER###

asap_data=[] #empty list which will contain our data

for i in range(len(os.listdir(ASAP_DIR))): #for each html file
    
    asap_df = {} #empty dictionary which will contain user data
    
    active_markets = []
    
    orders = 0 #set orders to 0 at beginning
    
    name = os.listdir(ASAP_DIR)[i][:-5] #vendor name
      
    
    with open(ASAP_DIR+'/' + os.listdir(ASAP_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        text = [i.lower() for i in list(txt)] #text as lowercase list
        
    #find active markets
    for market in list_of_markets:
        if market in soup.text.lower():
            active_markets.append(market)
        
        #get number of sales
        #first get number of sales on other markets
        for line in text:
            if market in line and market != 'asap' and len(line)<200: #only keep lines containing salees
                number = line.split()
                sales = number[-1].strip('():') #get number of sales for specified market
                try:
                    orders += int(sales) #check if it can be converted to an integer
                except:
                    continue
            else:
                continue
                
            #get number of sales in ASAP market
            if name in line: 
                home_sales = line
    try:
        orders += int(home_sales.split()[1].strip('():')) #add home orders to total orders
    except:
        continue

            
    
    #remove duplicates   
    active_markets = list(dict.fromkeys(active_markets))
       
    #add name, categories,completed orders, location and known markets
    asap_df['name'] = name
    asap_df['active_markets'] = active_markets
    asap_df['sales'] = orders
    
    asap_data.append(asap_df) #add user to dataframe
        
        
#         if categories != []:
#             df.loc[df['name'] == nam

In [249]:
asap_data = pd.DataFrame(data=asap_data)
asap_data.to_csv('data/asap_data.csv')
asap_data[asap_data.name == 'monoko']

,name,active_markets,sales
99,monoko,"[apollon, empire, asap, nightmare, wall street...",29769


In [250]:
###VICE_CITY SCRAPER###

vice_data=[] #empty list which will contain our data

for i in range(len(os.listdir(VICE_DIR))): #for each html file
    
    vice_df = {} #empty dictionary which will contain user data
    
    active_markets = []
    
    name = os.listdir(VICE_DIR)[i][:-5] #vendor name
      
    
    with open(VICE_DIR+'/' + os.listdir(VICE_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
    #find active markets
    for market in list_of_markets:
        if market in soup.text.lower():
            active_markets.append(market)
    
    #remove duplicates   
    active_markets = list(dict.fromkeys(active_markets))
       
    #add name, categories,completed orders, location and known markets
    vice_df['name'] = name
    vice_df['active_markets'] = active_markets
    
    vice_data.append(vice_df) #add user to dataframe

In [251]:
vice_data = pd.DataFrame(data=vice_data)
vice_data.to_csv('data/vice_data.csv')
vice_data.head(15)

,name,active_markets
0,fullzipp,[vice city]
1,k1to,[vice city]
2,mysteryland,[vice city]
3,CherryUK,[vice city]
4,Venture,[vice city]
5,mythbusters,[vice city]
6,shroomery,"[vice city, asap]"
7,TheWizzardNL,[vice city]
8,Crypto2Psychs,[vice city]
9,superhero,[vice city]


In [252]:
###SCRAPE DATA FROM SILKROAD####
from bs4 import BeautifulSoup as bs

silkroad_data=[] #empty list which will contain our data

for i in range(len(os.listdir(SILKROAD_DIR))): #for each html file
    
    silkroad_df = {} #empty dictionary which will contain user data
    
    categories = []
    ships_from = []
    
    name = os.listdir(SILKROAD_DIR)[i][:-5] #vendor name
      
    
    with open(SILKROAD_DIR+'/' + os.listdir(SILKROAD_DIR)[i]) as f: #open the file
        soup = bs(f, 'html.parser') #read using beautiful soup

        txt = soup.find_all(text=True) #only text
        
        
    #find product categories    
    for idx,line in enumerate(list(txt)):
        if 'Category' in line:
            category = txt[idx+1] #get text after 'category:'
            if category == 'Ships From:':
                pass
            else:
                categories.append(category)
            
        if 'Ships From' in line:
            ships = txt[idx+1]
            
            if ships == 'Not Specified':
                continue
                
            if ships != '\n':
                ships_from.append(ships)
            
            
            
    #remove duplicates  
    categories = list(dict.fromkeys(categories))
    ships_from = list(dict.fromkeys(ships_from))
    

    #add name, categories,completed orders, location and known markets
    silkroad_df['name'] = name
    silkroad_df['products'] = categories
    silkroad_df['ships_from'] = ships_from
    
    silkroad_data.append(silkroad_df) #add user to dataframe
    
silk_data = pd.DataFrame(data=silkroad_data)
silk_data.to_csv('data/silk_data.csv')
silk_data.head(15)

,name,products,ships_from
0,glasshouse,"[Weed, Concentrates, eBooks, PayPal, Edibles, ...",[United States]
1,mysteryland,"[Speed, Ecstasy]",[Germany]
2,AmsterdamLiquidXno1,"[LSD, 2C, Ketamine, Oxycodon, Ecstasy]",[Netherlands]
3,docspro,"[Scans, Cocaine, Carded Items, Xanax, Counterf...",[Ukraine]
4,LauraSilver,[Buds & Flowers],[Germany]
5,GRUENEWALDSCAVE,"[Ecstasy, Cocaine]",[United Kingdom]
6,alanmarret,"[Accounts, Banks, CC/CVV, Counterfeits, Carded...",[United States]
7,EmergencyB,[LSD],[Spain]
8,doneycc,[Ships From: ],[United States of America]
9,meds2buy,"[Oxycodon, Ecstasy]",[United States of America]


In [253]:
# build dataframe containing which markets the user is active
import pandas as pd
import numpy as np


test = []
for i in all_vendors:
    
    data = {}
    data['name'] = i
    data['markets'] = []
    data['ships_from'] = []
    data['products'] = []
    data['sales'] = 'NaN'
    sales = 0
    
    if i in silkroad_vendors:
        vendor_data = silk_data.loc[silk_data.name == i] #find same vendor in df
        shipping = vendor_data['ships_from'].iloc[0] #get shipping
        products = vendor_data['products'].iloc[0]
        
        
        data['markets'].append('silkroad')
        data['ships_from'].extend(shipping)
        data['products'].extend(products)
        
    if i in vice_vendors:
        vendor_data = vice_data.loc[vice_data.name == i] #find same vendor in df
        vendor_markets = vendor_data.active_markets.iloc[0] #get active markets
        
        data['markets'].extend(vendor_markets)

    if i in darkfox_vendors:
        vendor_data = darkfox_df.loc[darkfox_df.name == i] #find same vendor in df
        shipping = vendor_data['ships_from'].iloc[0] #get shipping
        products = vendor_data['products'].iloc[0] #add products
        
        if sales == 0: #make sure we dont add up the same order twice
            sales += vendor_data['orders'].iloc[0]
        
        
        data['markets'].append('darkfox')#add market
        data['ships_from'].extend(shipping) #add shipping
        data['products'].extend(products)
        
    if i in asap_vendors:
        vendor_data = asap_data.loc[asap_data.name == i] #find same vendor in df
        vendor_markets = vendor_data.active_markets.iloc[0] #get active markets
        
        data['markets'].extend(vendor_markets)
        
        if sales == 0:
            sales += vendor_data['sales'].iloc[0] 
 
        
    
    data['markets']=list(dict.fromkeys(data['markets'])) #remove duplicates
    data['ships_from']=list(dict.fromkeys(data['ships_from']))
    data['products']=list(dict.fromkeys(data['products']))
    data['sales']= sales
    data['n_markets'] = len(data['markets'])
    
    test.append(data)

df = pd.DataFrame(data=test)
df.sort_values('n_markets', ascending=False)

,name,markets,ships_from,products,sales,n_markets
693,PostNL,"[silkroad, apollon, white house, vice city, to...",[Netherlands],"[LSD, Heroin, Ecstasy, Ketamine]",1663,13
1083,MickeyMouseNLL,"[apollon, white house, vice city, torrez, empi...",[],[],0,13
786,XXXXTentacion,"[silkroad, apollon, white house, vice city, em...",[Netherlands],[Ships From: ],3131,12
818,SteroidWarehouse,"[silkroad, apollon, white house, empire, asap,...",[Netherlands],"[Steroids, Pills]",2300,12
1138,DutchDrugz,"[apollon, torrez, empire, asap, berlusconi, mo...",[],[],0,12
...,...,...,...,...,...,...
445,ProAusDrops,[darkfox],[],[],0,1
447,Naag,[darkfox],[],"[Videos & Photos, Fake IDs, E-Books, Services]",0,1
449,NotSoAverage,[darkfox],[United States],[Benzos],20,1
450,GreenLanternUK,[darkfox],[United Kingdom],"[Edibles, Weed]",7,1


In [255]:
df.to_csv('data/all_users.csv')